### Statistical Inference of Hidden Markov Models on High Frequency Quote Data

Benchmarking PSG performance of statistical inference against HMMLearn 

References:

PSG: http://aorda.com/html/PSG_Help_HTML/index.html?hmm_normal.htm

HMMLearn: https://hmmlearn.readthedocs.io/en/latest/index.html



In [155]:
import pandas as pd
import numpy as np
from hmmlearn.hmm import GaussianHMM
import matplotlib.pyplot as plt

import os
os.add_dll_directory('C:\Aorda\PSG\lib')
import psgpython as psg 
from psg_loader import load_psg

load_psg()

Inputting features removing duplicated values within each observation


In [207]:

features=pd.read_csv('data/grouped_features.csv',index_col=0)
frac=0.75


train_features=features[:np.floor(frac*features.shape[0]).astype(int)]
test_features=features[np.floor(frac*features.shape[0]).astype(int):]

def remove_duplicates(series):
    
    cleaned_series=series[np.insert(np.diff(series).astype(bool), 0, True)]
    dropped_els=len(series)-len(cleaned_series)
    
    print(f"Dropped {dropped_els} of original {len(series)} consecutive repeated values from input series")
    return cleaned_series

bidsize=remove_duplicates(train_features['Bid_Size'].values)
offersize=remove_duplicates(train_features['Offer_Size'].values)
bookimbalance=remove_duplicates(train_features['OB_IB'].values)
spread=remove_duplicates(train_features['spread'].values)

# formatted as numpy float 
np.savetxt(r'psg_text_hmm/vector_bidsize.txt', bidsize)
np.savetxt(r'psg_text_hmm/vector_offersize.txt', offersize)
np.savetxt(r'psg_text_hmm/vector_bookimbalance.txt', bookimbalance)
np.savetxt(r'psg_text_hmm/vector_spread.txt', spread)



Dropped 436 of original 15829 consecutive repeated values from input series
Dropped 679 of original 15829 consecutive repeated values from input series
Dropped 95 of original 15829 consecutive repeated values from input series
Dropped 697 of original 15829 consecutive repeated values from input series


In [208]:
train_features

,Bid_Size,Offer_Size,spread,OB_IB,last_spread,spread_state
sec,,,,,,
2020-01-02 09:30:00,1.142857,1.142857,0.241429,1.071429,0.589358,-1
2020-01-02 09:30:01,8.403670,7.559633,0.589358,2.472281,0.058205,1
2020-01-02 09:30:02,8.871795,5.051282,0.058205,1.111623,0.166977,-1
2020-01-02 09:30:03,3.604651,3.604651,0.166977,1.873784,1.260233,-1
2020-01-02 09:30:04,5.348837,1.860465,1.260233,0.971420,0.344615,1
...,...,...,...,...,...,...
2020-01-02 14:34:02,2.000000,24.000000,0.010000,12.000000,0.010000,0
2020-01-02 14:34:03,3.500000,14.500000,0.010000,12.833333,0.010000,0
2020-01-02 14:34:05,2.000000,5.000000,0.010000,2.500000,0.010000,0


# Steps

- Train HMM on one feature at a time
- Assume each feature is sampled according to two normal distributions that are our hidden states. 
- Learn optimal parameterization of hidden states

### PSG

Utilized HMM_Normal optimization routine for each feature

Initial point is estimated via Baum-Welch Algorithm

Inference is completed via constrained optimization



### Spread

In [209]:
psg_spread_prob = psg.psg_importfromtext('./psg_text_hmm/problem_hmm_normal_spread.txt')
psg_spread_prob['problem_statement'] = '\n'.join(psg_spread_prob['problem_statement'])
spread_solution=psg.psg_solver(psg_spread_prob)
spread_solution.values()

OK. Problem Imported

Running solver
Reading problem formulation
Asking for data information
Getting data
    100.0% of scenarios is processed
100% of vector_spread was read
Start optimization
Ext.iteration=0  Objective=0.746387610616E+00  Residual=0.000000000000E+00
Ext.iteration=10  Objective=0.746387610616E+00  Residual=0.000000000000E+00
Optimization is stopped
Solution is optimal
Calculating resulting outputs. Writing solution.
Objective: objective = 31117.1638666 [-4.342745230506E+16]
Solver has normally finished. Solution was saved.
Problem: problem_hmm_normal, solution_status = optimal
Timing: data_loading_time = 0.08, preprocessing_time = 15.12, solving_time = 1.51
Variables: optimal_point = point_problem_hmm_normal
Objective: objective = 31117.1638666 [-4.342745230506E+16]
Constraint: sum_of_probabilities_for_states = vector_sum_of_probabilities_for_states
Function: hmm_normal(2,vector_spread) =  3.111716386665E+04
OK. Solver Finished



dict_values(['problem_hmm_normal', 'optimal', ['problem_HMM_Normal, maximize', '  hmm_normal(2,vector_spread)', '  Solver: VAN , precision=9, stages=10'], ['Problem: problem_hmm_normal, solution_status = optimal', 'Timing: data_loading_time = 0.08, preprocessing_time = 15.12, solving_time = 1.51', 'Variables: optimal_point = point_problem_hmm_normal', 'Objective: objective = 31117.1638666 [-4.342745230506E+16]', 'Constraint: sum_of_probabilities_for_states = vector_sum_of_probabilities_for_states', 'Function: hmm_normal(2,vector_spread) =  3.111716386665E+04'], [['p1', 'p2', 'a1_1', 'a1_2', 'a2_1', 'a2_2', 'mu1', 'si1', 'mu2', 'si2'], array([0.        , 1.        , 0.92329517, 0.07670483, 0.56216745,
       0.43783255, 0.03386868, 0.01370098, 0.5525988 , 1.09801273])], array([2., 2., 1., ..., 1., 1., 1.]), array([1., 1., 1.]), array([0.00000000e+00, 1.17683641e-14, 6.43929354e-15]), [['state1', 'state2'], array([[0.        , 1.        ],
       [0.        , 1.        ],
       [0.80466

### Book Imbalance

In [210]:
psg_bookimbalance_prob = psg.psg_importfromtext('./psg_text_hmm/problem_hmm_normal_bookimbalance.txt')
psg_bookimbalance_prob['problem_statement'] = '\n'.join(psg_bookimbalance_prob['problem_statement'])
bookimbalance_solution=psg.psg_solver(psg_bookimbalance_prob)
bookimbalance_solution.values()

OK. Problem Imported

Running solver
Reading problem formulation
Asking for data information
Getting data
    100.0% of scenarios is processed
100% of vector_bookimbalance was read
Start optimization
Ext.iteration=0  Objective=0.550280448269E+00  Residual=0.000000000000E+00
Ext.iteration=17  Objective=0.550280448269E+00  Residual=0.000000000000E+00
Ext.iteration=33  Objective=0.550280448269E+00  Residual=0.000000000000E+00
Ext.iteration=50  Objective=0.550280448269E+00  Residual=0.000000000000E+00
Ext.iteration=65  Objective=0.550280448269E+00  Residual=0.000000000000E+00
Ext.iteration=81  Objective=0.550280448269E+00  Residual=0.000000000000E+00
Ext.iteration=98  Objective=0.550280448269E+00  Residual=0.000000000000E+00
Ext.iteration=112  Objective=0.550280448269E+00  Residual=0.000000000000E+00
Ext.iteration=131  Objective=0.550280448269E+00  Residual=0.000000000000E+00
Ext.iteration=144  Objective=0.550280448269E+00  Residual=0.000000000000E+00
Optimization is stopped
Solution is fe

dict_values(['problem_hmm_normal', 'feasible', ['problem_HMM_Normal, maximize', '  hmm_normal(2,vector_bookimbalance)', '  Solver: VAN , precision=9, stages=10'], ['Problem: problem_hmm_normal, solution_status = feasible', 'Timing: data_loading_time = 0.09, preprocessing_time = 15.46, solving_time = 18.52', 'Variables: optimal_point = point_problem_hmm_normal', 'Objective: objective = -19076.7008834 [-0.433340420857]', 'Constraint: sum_of_probabilities_for_states = vector_sum_of_probabilities_for_states', 'Function: hmm_normal(2,vector_bookimbalance) = -1.907670088342E+04'], [['p1', 'p2', 'a1_1', 'a1_2', 'a2_1', 'a2_2', 'mu1', 'si1', 'mu2', 'si2'], array([1.        , 0.        , 0.96627022, 0.03372978, 0.48776439,
       0.51223561, 1.27015439, 0.61783865, 5.26742548, 5.86678499])], array([1., 1., 1., ..., 1., 1., 2.]), array([1., 1., 1.]), array([0.00000000e+00, 1.33226763e-14, 9.32587341e-15]), [['state1', 'state2'], array([[1.00000000e+00, 0.00000000e+00],
       [9.88620821e-01, 1.

### Offer Size

In [211]:
psg_offersize_prob = psg.psg_importfromtext('./psg_text_hmm/problem_hmm_normal_offersize.txt')
psg_offersize_prob['problem_statement'] = '\n'.join(psg_offersize_prob['problem_statement'])
offersize_solution=psg.psg_solver(psg_offersize_prob)
offersize_solution.values()

OK. Problem Imported

Running solver
Reading problem formulation
Asking for data information
Getting data
    100.0% of scenarios is processed
100% of vector_offersize was read
Start optimization
Ext.iteration=0  Objective=0.640663347363E+00  Residual=0.000000000000E+00
Ext.iteration=47  Objective=0.640663347363E+00  Residual=0.000000000000E+00
Ext.iteration=94  Objective=0.640663347363E+00  Residual=0.000000000000E+00
Ext.iteration=142  Objective=0.640663347364E+00  Residual=0.000000000000E+00
Ext.iteration=144  Objective=0.640663347364E+00  Residual=0.000000000000E+00
Optimization is stopped
Solution is feasible
Calculating resulting outputs. Writing solution.
Objective: objective = -22506.9342039 [-0.439133406798]
Solver has normally finished. Solution was saved.
Problem: problem_hmm_normal, solution_status = feasible
Timing: data_loading_time = 0.05, preprocessing_time = 8.04, solving_time = 6.34
Variables: optimal_point = point_problem_hmm_normal
Objective: objective = -22506.9342

dict_values(['problem_hmm_normal', 'feasible', ['problem_HMM_Normal, maximize', '  hmm_normal(2,vector_offersize)', '  Solver: VAN , precision=9, stages=10'], ['Problem: problem_hmm_normal, solution_status = feasible', 'Timing: data_loading_time = 0.05, preprocessing_time = 8.04, solving_time = 6.34', 'Variables: optimal_point = point_problem_hmm_normal', 'Objective: objective = -22506.9342039 [-0.439133406798]', 'Constraint: sum_of_probabilities_for_states = vector_sum_of_probabilities_for_states', 'Function: hmm_normal(2,vector_offersize) = -2.250693420394E+04'], [['p1', 'p2', 'a1_1', 'a1_2', 'a2_1', 'a2_2', 'mu1', 'si1', 'mu2', 'si2'], array([0.        , 1.        , 0.96486587, 0.03513413, 0.37422167,
       0.62577833, 2.1656628 , 0.75128616, 7.89900263, 8.40985197])], array([2., 2., 2., ..., 2., 2., 2.]), array([1., 1., 1.]), array([1.82853732e-13, 1.19904087e-14, 3.55271368e-15]), [['state1', 'state2'], array([[0.00000000e+00, 1.00000000e+00],
       [3.33684060e-12, 1.00000000e+

### Bid Size

In [212]:
psg_bidsize_prob = psg.psg_importfromtext('./psg_text_hmm/problem_hmm_normal_bidsize.txt')
psg_bidsize_prob['problem_statement'] = '\n'.join(psg_bidsize_prob['problem_statement'])
bidsize_solution=psg.psg_solver(psg_bidsize_prob)
bidsize_solution.values()

OK. Problem Imported

Running solver
Reading problem formulation
Asking for data information
Getting data
    100.0% of scenarios is processed
100% of vector_bidsize was read
Start optimization
Ext.iteration=0  Objective=0.573682602654E+00  Residual=0.000000000000E+00
Ext.iteration=50  Objective=0.573682602654E+00  Residual=0.000000000000E+00
Ext.iteration=99  Objective=0.573682602654E+00  Residual=0.000000000000E+00
Ext.iteration=144  Objective=0.573682602654E+00  Residual=0.000000000000E+00
Optimization is stopped
Solution is feasible
Calculating resulting outputs. Writing solution.
Objective: objective = -22767.6679717 [-0.496085898942]
Solver has normally finished. Solution was saved.
Problem: problem_hmm_normal, solution_status = feasible
Timing: data_loading_time = 0.07, preprocessing_time = 8.03, solving_time = 6.14
Variables: optimal_point = point_problem_hmm_normal
Objective: objective = -22767.6679717 [-0.496085898942]
Constraint: sum_of_probabilities_for_states = vector_sum_

dict_values(['problem_hmm_normal', 'feasible', ['problem_HMM_Normal, maximize', '  hmm_normal(2,vector_bidsize)', '  Solver: VAN , precision=9, stages=10'], ['Problem: problem_hmm_normal, solution_status = feasible', 'Timing: data_loading_time = 0.07, preprocessing_time = 8.03, solving_time = 6.14', 'Variables: optimal_point = point_problem_hmm_normal', 'Objective: objective = -22767.6679717 [-0.496085898942]', 'Constraint: sum_of_probabilities_for_states = vector_sum_of_probabilities_for_states', 'Function: hmm_normal(2,vector_bidsize) = -2.276766797166E+04'], [['p1', 'p2', 'a1_1', 'a1_2', 'a2_1', 'a2_2', 'mu1', 'si1', 'mu2', 'si2'], array([0.        , 1.        , 0.94899043, 0.05100957, 0.39987896,
       0.60012104, 2.24465004, 0.72511467, 5.76143998, 4.8864252 ])], array([2., 2., 2., ..., 1., 1., 1.]), array([1., 1., 1.]), array([0.00000000e+00, 7.32747196e-15, 4.66293670e-15]), [['state1', 'state2'], array([[0.00000000e+00, 1.00000000e+00],
       [9.52712471e-17, 1.00000000e+00],

### HMM Learn


- Set 2 hidden components
- Solves via the Viterbi Forward Backwards Algorithm
- Full covariance matrix with min_covar to prevent overfitting
- Tolerance set equivantly to PSG

### Spread

In [213]:
spread_model=GaussianHMM(n_components=2,algorithm='viterbi',covariance_type="spherical",min_covar=1e-4, n_iter=1000,tol=1e-8, verbose=True)
fitted_spread_model=spread_model.fit(spread.reshape(-1, 1))

         1      -74496.5604             +nan
         2       11313.9291      +85810.4895
         3       21624.1755      +10310.2464
         4       27094.1741       +5469.9986
         5       29533.0020       +2438.8279
         6       30497.0739        +964.0719
         7       30894.9109        +397.8370
         8       31041.5028        +146.5919
         9       31091.0698         +49.5670
        10       31107.7729         +16.7031
        11       31113.5686          +5.7957
        12       31115.6614          +2.0928
        13       31116.4565          +0.7951
        14       31116.7786          +0.3221
        15       31116.9190          +0.1404
        16       31116.9849          +0.0659
        17       31117.0179          +0.0330
        18       31117.0353          +0.0174
        19       31117.0448          +0.0095
        20       31117.0500          +0.0053
        21       31117.0530          +0.0030
        22       31117.0547          +0.0017
        23

In [214]:
print(f"Transition Matrix is {fitted_spread_model.transmat_.flatten()}")
print(f"Mean Values are is {fitted_spread_model.means_.flatten()}")
print(f"Covariance Matrix is {fitted_spread_model.covars_.flatten()}")

Transition Matrix is [0.92348119 0.07651881 0.56258625 0.43741375]
Mean Values are is [0.03388608 0.55392045]
Covariance Matrix is [1.89240669e-04 1.20837905e+00]


### Book Imbalance

In [215]:
bookimbalance_model=GaussianHMM(n_components=2,algorithm='viterbi',covariance_type="spherical",min_covar=1e-4, n_iter=1000,tol=1e-8, verbose=True)
fitted_bookimbalance_model=bookimbalance_model.fit(bookimbalance.reshape(-1, 1))

         1      -36170.8174             +nan
         2      -21928.1596      +14242.6579
         3      -19815.2928       +2112.8668
         4      -19287.1485        +528.1443
         5      -19149.2612        +137.8873
         6      -19101.5593         +47.7019
         7      -19085.1258         +16.4334
         8      -19079.5451          +5.5807
         9      -19077.6607          +1.8844
        10      -19077.0251          +0.6356
        11      -19076.8105          +0.2145
        12      -19076.7380          +0.0725
        13      -19076.7135          +0.0245
        14      -19076.7052          +0.0083
        15      -19076.7023          +0.0028
        16      -19076.7014          +0.0010
        17      -19076.7011          +0.0003
        18      -19076.7009          +0.0001
        19      -19076.7009          +0.0000
        20      -19076.7009          +0.0000
        21      -19076.7009          +0.0000
        22      -19076.7009          +0.0000
        23

In [216]:
print(f"Transition Matrix is {fitted_bookimbalance_model.transmat_.flatten()}")
print(f"Mean Values are is {fitted_bookimbalance_model.means_.flatten()}")
print(f"Covariance Matrix is {fitted_bookimbalance_model.covars_.flatten()}")

Transition Matrix is [0.96627036 0.03372964 0.48776422 0.51223578]
Mean Values are is [1.27015475 5.26743463]
Covariance Matrix is [ 0.3817259  34.41927375]


### Bid Size

In [217]:
bidsize_model=GaussianHMM(n_components=2,algorithm='viterbi',covariance_type="spherical",min_covar=1e-4, n_iter=1000,tol=1e-8, verbose=True)
fitted_bidsize_model=bidsize_model.fit(bidsize.reshape(-1, 1))

         1      -37020.3760             +nan
         2      -24164.7227      +12855.6532
         3      -23358.2629        +806.4598
         4      -23037.8171        +320.4458
         5      -22901.8110        +136.0061
         6      -22836.5719         +65.2391
         7      -22802.8371         +33.7348
         8      -22785.4744         +17.3627
         9      -22776.8232          +8.6512
        10      -22772.6350          +4.1882
        11      -22770.6330          +2.0020
        12      -22769.6396          +0.9933
        13      -22769.0270          +0.6126
        14      -22768.4863          +0.5407
        15      -22768.0436          +0.4427
        16      -22767.8065          +0.2370
        17      -22767.7153          +0.0912
        18      -22767.6844          +0.0309
        19      -22767.6739          +0.0104
        20      -22767.6703          +0.0037
        21      -22767.6689          +0.0014
        22      -22767.6684          +0.0005
        23

In [218]:
print(f"Transition Matrix is {fitted_bidsize_model.transmat_.flatten()}")
print(f"Mean Values are is {fitted_bidsize_model.means_.flatten()}")
print(f"Covariance Matrix is {fitted_bidsize_model.covars_.flatten()}")

Transition Matrix is [0.94899063 0.05100937 0.3998793  0.6001207 ]
Mean Values are is [2.24465083 5.76144924]
Covariance Matrix is [ 0.5257934  23.87723822]


### Offer Size

In [219]:
offersize_model=GaussianHMM(n_components=2,algorithm='viterbi',covariance_type="spherical",min_covar=1e-4, n_iter=1000,tol=1e-8, verbose=True)
fitted_offersize_model=offersize_model.fit(offersize.reshape(-1, 1))

         1     -100805.2281             +nan
         2      -26409.6368      +74395.5913
         3      -23642.1305       +2767.5063
         4      -22801.6824        +840.4481
         5      -22592.3205        +209.3619
         6      -22534.3155         +58.0050
         7      -22516.4261         +17.8894
         8      -22510.6892          +5.7369
         9      -22508.8512          +1.8380
        10      -22508.2678          +0.5834
        11      -22508.0840          +0.1838
        12      -22508.0264          +0.0576
        13      -22508.0084          +0.0180
        14      -22508.0028          +0.0056
        15      -22508.0011          +0.0018
        16      -22508.0005          +0.0005
        17      -22508.0004          +0.0002
        18      -22508.0003          +0.0001
        19      -22508.0003          +0.0000
        20      -22508.0003          +0.0000
        21      -22508.0003          +0.0000
        22      -22508.0003          +0.0000
        23

In [220]:
print(f"Transition Matrix is {fitted_offersize_model.transmat_.flatten()}")
print(f"Mean Values are is {fitted_offersize_model.means_.flatten()}")
print(f"Covariance Matrix is {fitted_offersize_model.covars_.flatten()}")

Transition Matrix is [0.96476318 0.03523682 0.37490105 0.62509895]
Mean Values are is [2.16556749 7.9049774 ]
Covariance Matrix is [ 0.56440982 70.746429  ]


### Stationary Distributions

Limiting marginal distributions

In [221]:
spread_stationary=fitted_spread_model.get_stationary_distribution()
bookimbalance_stationary=fitted_bookimbalance_model.get_stationary_distribution()
bidsize_stationary=fitted_bidsize_model.get_stationary_distribution()
offersize_stationary=fitted_offersize_model.get_stationary_distribution()

print(f"Stationary Distribution for Spread HMM is {spread_stationary}")
print(f"Stationary Distribution for Book Imbalance HMM is {bookimbalance_stationary}")
print(f"Stationary Distribution for Bidsize HMM is {bidsize_stationary}")
print(f"Stationary Distribution for Offersize HMM is {offersize_stationary}")

Stationary Distribution for Spread HMM is [0.88027194 0.11972806]
Stationary Distribution for Book Imbalance HMM is [0.93532112 0.06467888]
Stationary Distribution for Bidsize HMM is [0.88686926 0.11313074]
Stationary Distribution for Offersize HMM is [0.91408543 0.08591457]


### Out of Sample Validation and Scoring

Given a test set we can score performance of PSG trained model and HMMLearn trained Model

In [228]:
spread_pred=fitted_spread_model.predict(test_features['spread'].values.reshape(-1, 1))


In [224]:
bookimbalance_pred=fitted_bookimbalance_model.predict(test_features['OB_IB'].values.reshape(-1, 1))

In [226]:
bidsize_pred=fitted_bidsize_model.predict(test_features['Bid_Size'].values.reshape(-1, 1))

In [227]:
offersize_pred=fitted_offersize_model.predict(test_features['Offer_Size'].values.reshape(-1, 1))

Actual evaluation can be computed as a T test on the sample coming from either distribution. Reject or do not reject at $\alpha$ confidence level
